<a href="https://colab.research.google.com/github/Leo-2017/DL101/blob/main/Pattern_Player.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pattern Player 🎹

## **1. Introduction**

## a. What business problem are you solving?

A simple tool for non-design professionals to design wallpaper for their homes by themselves.

## b. How can AI/ML technology help to solve that?

The AI/ML technology can help to slove it by using the GPT-2 model (pretrained on a dataset of 8 million web pages) as a foundation, and using dataset from Gutenberg Project to fine-tune the model using Text Generation to make it more applicable for the use case.

## c. What is your AI project's objective?

The objective is to build a language model as a pattern design generator that helps user to predict the probability of what kind of wallpaper user would like to have.

## d. Who are the users of your AI application and how will they interact with it? 

The user of the AI application will be the non-design professionals who have a needs in design & decorating the wallpaper at their e.g. homes by themselves; 
<br>
<br>
User can simply input the **keywords** or **short description** such as **"How to decorate the wall in living room with Botanical pattern in william morris style?"** The application will predict the output as a suggestion based on the dataset;

## **2. Benchmark Study**

**Similar existing AI solutions** <br>

Below are the pattern generator I found online; for most of them the way how it works is to: <br>
A. require user to upload an asset (image/illustration, etc.), the application will generator a pattern based on it; or <br>
B. Let user pre-select the pattern from their dataset and then upload their homeliving photo to see the model; 

**Picsart** <br>
https://picsart.com/pattern-generator <br>

**Patternico** <br>
https://patternico.com/

**Wowpatterns** <br>
https://www.wowpatterns.com/pattern-generator

**Laurentvw** <br>
https://www.laurentvw.com/tools/seamless-pattern-generator/

**Designhill** <br>
https://www.designhill.com/tools/wallpaper-maker

**Patterncooler** <br>
https://www.patterncooler.com/

**Astreetprints** <br>
https://www.astreetprints.com/room-previewer

---
All the applications I could find above is not exactly what I'm looking for; <br>
Below is the usecase which is very similar with what I would like to build for my model:

**Delle-mini** <br> 
https://huggingface.co/spaces/dalle-mini/dalle-mini <br>
https://www.craiyon.com/ <br>
🟦
<img src="https://drive.google.com/file/d/1fgQF5qRt3ho4Mdtiz8k6S14Y3RAo7GV3/view?usp=sharing" alt="Logo" width="100"/>
<img src="https://drive.google.com/file/d/1Sy2Zzs8vZOjtu3WeWhsC7B-V5r6cHm4V/view?usp=sharing" alt="dallemini" width="300"/> <br>

---
Due to the limited time and my limited knowledges for the MVP I will only build a text generator;

# **3. Data**

## a. Libraries & Dependencies needed

In [ ]:
!pip install -Uqq unpackai
# !pip install -q git+https://github.com/unpackai/unpackai
!pip install -Uqq transformers==4.10.2
!pip install -q datasets transformers[sentencepiece]

     |████████████████████████████████| 73 kB 906 kB/s 
     |████████████████████████████████| 42 kB 595 kB/s 
     |████████████████████████████████| 976 kB 11.9 MB/s 
     |████████████████████████████████| 2.8 MB 5.1 MB/s 
     |████████████████████████████████| 880 kB 49.9 MB/s 
     |████████████████████████████████| 101 kB 9.2 MB/s 
     |████████████████████████████████| 596 kB 45.7 MB/s 
     |████████████████████████████████| 3.3 MB 35.1 MB/s 
     |████████████████████████████████| 362 kB 5.3 MB/s 
     |████████████████████████████████| 140 kB 10.5 MB/s 
     |████████████████████████████████| 212 kB 41.6 MB/s 
     |████████████████████████████████| 1.1 MB 33.4 MB/s 
     |████████████████████████████████| 127 kB 48.3 MB/s 
     |████████████████████████████████| 144 kB 58.9 MB/s 
     |████████████████████████████████| 271 kB 35.7 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 1.1 MB 41.7 MB/s 
ERROR: pip's dependency

In [ ]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModel,
    pipeline,
    set_seed,
    Trainer,
    TextDataset,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    AutoModelForSequenceClassification,
)
from unpackai.nlp import Textual, InterpEmbeddingsTokenizer
from ipywidgets import interact
import logging
from fastai import *

## 2. Build an Language Model

### a. What's your data? Is it relevant to AI project you are intending to build? Is it inclusive, complete and unbiased?


#### The dataset I'm using for this model training is collected from [Gutenberg Project](https://gutenberg.org/); a source which is a collection of over 60,000 free eBooks.
**Dataset:** 
<br>
1. [Arts and Crafts Essays by members of Arts and Crafts Exhibition Society](https://gutenberg.org/ebooks/36250) Subject: Decorative arts <br>
2. [Principles of Decorative Design by Christopher Dresser](https://gutenberg.org/ebooks/39749) Subject: Decorative arts Decoration and ornament -- Victorian style<br>
3. [The Botanical Magazine, Vol. 02 by William Curtis](https://gutenberg.org/ebooks/17531) Subject: Plants, Cultivated/Ornamental/ Botanical illustration/Periodicals <br>
4. [The Principles of Ornament by James Ward](https://gutenberg.org/ebooks/60034) Subject: Decoration and ornament <br>
5. [The Botanical Magazine, Vol. 08 by William Curtis](https://gutenberg.org/ebooks/24670) Subject: Plants, Cultivated/Ornamental/ Botanical illustration/Periodicals
6. [Evolution in Art: As Illustrated by the Life-histories of Designs by Haddon](https://gutenberg.org/ebooks/46079) Subject: Decoration and ornament, Primitive <br>
7. [Hardy Ornamental Flowering Trees and Shrubs by Angus D. Webster](https://gutenberg.org/ebooks/10852) Subject: Flowering shrubs <br>
8. [Origin and Development of Form and Ornament in Ceramic Art. by William Henry Holmes](https://gutenberg.org/ebooks/19953) Subject: 	Decoration and ornament -- History, Indian art -- North America <br>
9. [The Botanical Magazine, Vol. 04 by William Curtis](https://gutenberg.org/ebooks/17979) Subject: Plants, Cultivated/Ornamental/ Botanical illustration/Periodicals <br>
10.  [The Botanical Magazine, Vol. 03 by William Curtis](https://gutenberg.org/ebooks/17672) Subject: Plants, Cultivated/Ornamental/ Botanical illustration/Periodicals <br>
11.  [The Botanical Magazine, Vol. 05 by William Curtis](https://gutenberg.org/ebooks/19123) Subject: Plants, Cultivated/Ornamental/ Botanical illustration/Periodicals <br>
12.  [The Botanical Magazine, Vol. 06 by William Curtis](https://gutenberg.org/ebooks/21843) Subject: Plants, Cultivated/Ornamental/ Botanical illustration/Periodicals <br>
13.  [The Botanical Magazine, Vol. 07 by William Curtis](https://gutenberg.org/ebooks/23579) Subject: Plants, Cultivated/Ornamental/ Botanical illustration/Periodicals <br>
14.  [The Botanical Magazine, Vol. 09 by William Curtis](https://gutenberg.org/ebooks/38382) Subject: Plants, Cultivated/Ornamental/ Botanical illustration/Periodicals <br>
15.  [The Botanical Magazine, Vol. 13 by William Curtis and John Sims](https://gutenberg.org/ebooks/43858) Subject: Plants, Cultivated/Ornamental/ Botanical illustration/Periodicals <br>
16.  [Talks About Flowers. by Mrs. M. D. Wellcome](https://gutenberg.org/ebooks/40534) Subject: Flowers <br>
17.  [Garden Ornaments by Mary Harrod Northend](https://gutenberg.org/ebooks/34885) Subject: Gardens <br>
18.  [link text](https://gutenberg.org/ebooks/33955) Subject: Gardens <br>
19.  [Remodeled Farmhouses by Mary Harrod Northend](https://) Subject: Farmhouses -- United States (Not sure about this data)<br>
20.  [Historic Ornament, Vol. 1 (of 2) by James Ward](https://gutenberg.org/ebooks/59746) Subject: Decorative arts -- History<br>
21.  [Historic Ornament, Vol. 2 (of 2) by James Ward](https://gutenberg.org/ebooks/59971) Subject: Decorative arts -- History<br>
22.  [Hardy Ornamental Flowering Trees and Shrubs by Angus D. Webster](https://gutenberg.org/ebooks/10852) Subject: Flowering shrubs <br>
---

#### Most of the books collected are relevant to AI project I'm intending to build; As if it's inclusive, complete and unbiased, I cannot genature as many books I've found online are not Free of use; Therefore I have included all the books/dataset I could find from  Gutenberg;

### b. Collect and construct your dataset

In order to collect and design my dataset I'm using `Textual` scraping tool below.

**Collect text from an URL.**


Below are all the URL I've collected from [Gutenberg Project] https://gutenberg.org/; a source which is a collection of over 60,000 free eBooks. 


In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/36250/pg36250.txt")
textual

Text (393777 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/39749/pg39749.txt")
textual

Text (414489 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/17531/pg17531.txt")
textual

Text (80332 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/files/60034/60034-0.txt")
textual

Text (234063 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/24670/pg24670.txt")
textual

Text (103125 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/files/46079/46079-0.txt")
textual

Text (667372 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/10852/pg10852.txt")
textual

Text (370515 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/19953/pg19953.txt")
textual

Text (72466 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/17979/pg17979.txt")
textual

Text (85944 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/17672/pg17672.txt")
textual

Text (76767 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/19123/pg19123.txt")
textual

Text (83010 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/21843/pg21843.txt")
textual

Text (93266 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/23579/pg23579.txt")
textual

Text (92322 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/38382/pg38382.txt")
textual

Text (92729 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/43858/pg43858.txt")
textual

Text (100458 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/40534/pg40534.txt")
textual

Text (305686 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/34885/pg34885.txt")
textual

Text (200928 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/33955/pg33955.txt")
textual

Text (343848 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/files/59746/59746-0.txt")
textual

Text (439720 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/files/59971/59971-0.txt")
textual

Text (522737 chars), textual(),
    train_path, val_path = textual.create_train_val()

In [ ]:
textual = Textual.from_url("https://gutenberg.org/cache/epub/10852/pg10852.txt")
textual

Text (370515 chars), textual(),
    train_path, val_path = textual.create_train_val()

### c. Is the size of your dataset large enough to build the baseline model to validate the usefulness and accuracy of the model predictions?


I have collected 22 books to build the baseline model; 


### **4. Model Training**

We skip the Data Transformation as the Tokenization and Numericalisation happens during the Model Training.

In [ ]:
pretrained_model = pipeline("text-generation", model='gpt2')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

### **5. Interpret the model and generate text.**

In [ ]:
arguments = TrainingArguments(
    output_dir="./write_style",
    overwrite_output_dir=True,  
    num_train_epochs=3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=64,
    )

In [ ]:
trainer = textual.get_hf_trainer(
    model=pretrained_model.model,
    tokenizer = pretrained_model.tokenizer,
    arguments = arguments
    )

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (80564 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 629
  Num Epochs = 3
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 81


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=81, training_loss=3.323629497010031, metrics={'train_runtime': 3522.504, 'train_samples_per_second': 0.536, 'train_steps_per_second': 0.023, 'total_flos': 123264516096000.0, 'train_loss': 3.323629497010031, 'epoch': 3.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to ./write_style
Configuration saved in ./write_style/config.json
Model weights saved in ./write_style/pytorch_model.bin


In [ ]:
style_writer = pipeline('text-generation',
                        model='./write_style',
                        tokenizer=pretrained_model.tokenizer
                        )

loading configuration file ./write_style/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": true,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "max_length": 50,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.10.2",
  "use_cache": true,
  "vocab_size": 50257
}

loading configurati

In [ ]:
style_writer("How to design botanical pattern?",
            max_length=150,
            num_return_sequences=1)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'How to design botanical pattern? This is the first book that directly addresses this subject.  This introduction provides a more technical, more elegant explanation than the usual work of the botanists, and, with its many references, is perhaps best produced in a form which will be freely available to the general public.  While the general method of the manufacture of flowers may be described in more detail elsewhere, in detail a description of how one plants leaves flowers with respect to their parent or object must still be given.  Although many useful tips and exercises are found in the preceding volumes, they should not be neglected if a suitable system are to be developed.  This book is being written for general purposes, not suited for those of ornamental design,'}]

In [ ]:
style_writer("How to design pattern in william morris style?",
            max_length=150,
            num_return_sequences=1)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "How to design pattern in william morris style? Flowers, with white heads and yellow-white berries, are produced from December till April in these beautiful forms. Each shoots four feet high, and grows from about one foot deep. They are very valuable early-growing varieties, and produced not only at our garden plants, but as also being of great value in the making of decorative shrubs, and in the growing centre of the walls of houses.  WILLIAM HUGBETTS (_syn Japonica_).--Common Willingham's Cabbage. North America. 2 vols., London. 416 or 417 c.  WILLIAM HUGBETTS (_syn Japonica_).--Japanese Green"}]

In [ ]:
style_writer("How to decorate the wall in living room with Botanical pattern in william morris style?",
            max_length=150,
            num_return_sequences=1)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'How to decorate the wall in living room with Botanical pattern in william morris style?  The only practical way to do this is by hanging it onto a wall. The larger the wall, the more interesting the flowers will become.  For the younger generations, these flowers should be planted to the inside of a window sill or hanging box.  The decorative flowers are arranged in clusters. The leaves of the flower-bearing flowers are produced in clusters.  Flowers that are larger are usually well-known shrubs like The Botanical Nursery of Wales, or the Almond Tree of the Isles, where they are produced in clusters.   A large number of ornamental flowers usually fall into the branches of this family, but not'}]

### **6. Model's input and output**

Your ML model's output comes back to what machine learning task you initially selected. Supervised machine learning tasks require labeled datasets (x and y) for the ML model to learn how to map your data (x) to the desired output/label (y).
Please describe your data labels (you can also insert the actual example of the label) and discuss if the model output satisfies the project's objective. 

What data type and format (e.g. jpeg images, csv file, pdf file) will you utilize for data pre-processing before you feed them for model training? <br>
The data type and format I'm using for the pre-processing is .txt file; <br>
x="How to design botanical pattern?" <br>
x="How to design pattern in william morris style?" <br>
x="How to decorate the wall in living room with Botanical pattern in william morris style?" <br>


What's your data label? What do you want to predict? <br>
I want to predict that the trained text geneator can answer user's questions or predict what they're looking for by user's inputed keywords; <br>
y= "max_length=150"

### 7. What metrics explain your model performance?

First think in terms of "Business Metrics" or business/practical objectives, not technical metrics. Then apply this thinking to figuring out one or a combination of ML evaluation metrics and human readable metrics that match the business objectives. <br>
For example, for an image classification model to evaluate its prediction results you could use error rate or accuracy, while confusion matrix can be better in explaining how model performs on a particular class. <br>
To learn about the metrics for NLP-related problems, please check: Hugging Face <br>
For general classification and regression problems, please check: neptune.ai's blog <br>

1. What are non-ML baselines you can consider as reference for a model performance?

2. What are your model evaluation metrics and why?

Metrics: <br>
bleurt <br>
https://huggingface.co/spaces/evaluate-metric/bleurt <br>
bert_score <br>
https://huggingface.co/spaces/evaluate-metric/bertscore <br>
glue <br>
https://huggingface.co/spaces/evaluate-metric/glue

why?

Is this sentence syntactically correct?

3. Is the metric you selected intuitive enough to communicate the results of the model training to non-ML guys? <br>
I'm not sure as I think the result so far is not bad, but it can be better with more data;

4. Can your metric explain model's biases? <br>
It defintelly can, as the data I've been feeding only covers certain styles of the design; more comprehensive data is needed to make the output more neutral and no biases;
